In [1]:
#using Pkg
#Pkg.add("PrettyTables")
using JSON
using Random
using Distributions
using MySQL
using DotEnv
using DataFrames
using PrettyTables
cfg = DotEnv.config(path="../.env")
const DB = DBInterface
include("/home/peters/code/mapinator/estimation/functions.jl")
YEAR_INTERVAL = 2003:2023  # change this to select the years of data to include in the estimation
NUM = 5 # variable set it here
NUMSINKS = 6 #4 sinks are assumed in some of the code
sinks = ["Other Academic","Government","Private Sector","Teaching Universities","Postdoc", "Sessionals","Column Totals"]
function name_to_oid(name, institution_mapping)
    ## returns an institution_id
    for (key, value) in institution_mapping
        if value == name
            return key
        end
    end
    return "not found"
end

name_to_oid (generic function with 1 method)

bucket_extract

In [2]:
 academic_builder, sink_builder, academic, acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer, institution_mapping = functions.process_data(YEAR_INTERVAL);

Debug: 14876 returned from api
Debug: 7273 academic placements
Debug: 7603 sinks
Debug: 14876 in all


In [3]:
println(length(academic_builder), " total assistant professor outcomes")
    println(length(sink_builder), " other outcomes")
    sample_size = length(academic_builder)+length(sink_builder)
    println(sample_size, " total outcomes")
println(length(acd_sink), " Other academic institutions")
println(length(gov_sink), " Government institutions")
println(length(pri_sink), " Private Sector institutions")
println(length(tch_sink), " Teaching institutions")
println(length(postdoc), " Institutions hiring postdocs")
println(length(lecturer), " Institutions hiring sessionals")


7273 total assistant professor outcomes
7603 other outcomes
14876 total outcomes
553 Other academic institutions
155 Government institutions
195 Private Sector institutions
627 Teaching institutions
514 Institutions hiring postdocs
366 Institutions hiring sessionals


In [4]:
institutions, out = functions.create_adjacency_matrix(academic_builder, sink_builder, academic,acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer);

In [ ]:
est_obj, est_alloc = functions.doit(out, collect(academic),collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink), collect(postdoc), collect(lecturer),institutions, NUM, NUMSINKS);

In [ ]:
est_mat, est_count = functions.bucket_extract(est_alloc, out, NUM, NUMSINKS)
M = est_mat

In [ ]:
t_table, t_allocation = functions.tiered_allocation(M, est_count, institutions, est_alloc, NUM, NUMSINKS);
t_table

In [ ]:
table_latex = functions.nice_table(t_table, NUM, NUMSINKS, sinks)
table_latex

In [ ]:
#for line in institution_mapping
#    println(line)
#end#

In [ ]:
d = DB.connect(MySQL.Connection,cfg["host"], cfg["user"], cfg["password"], db =cfg["database"], port = parse(Int64,cfg["port"]));
query = "select i.name,j.institution_id ,
SUM(CASE
  WHEN j.estimated_tier = 1 THEN count ELSE 0 END
) AS TIER1,
SUM(CASE
  WHEN j.estimated_tier = 2 THEN count ELSE 0 END
) AS TIER2,
SUM(CASE
  WHEN j.estimated_tier = 3 THEN count ELSE 0 END
) AS TIER3,
SUM(CASE
  WHEN j.estimated_tier = 4 THEN count ELSE 0 END
) AS TIER4,
SUM(CASE
  WHEN j.estimated_tier = 5 THEN count ELSE 0 END
) AS TIER5


from ftrecruiter.institutions i join
(select d.institution_id, d.estimated_tier, count(*) as count from bootstrap_data d left join bootstrap_trials t  
on t.trial_id=d.trial_id where t.number_of_tiers=?
group by institution_id, estimated_tier) j on i.institution_id=j.institution_id where j.institution_id=?
group by j.institution_id"
get_statement =  DB.prepare(d, query);


In [39]:
function get_confidence(tiers, institution_id, get_statement)
    m = DB.execute(get_statement, [tiers, institution_id]) |> DataFrame
    return m
    if nrow(m) > 0
        r = first(m)
        t = 0
        s = 0
        s+= r.TIER1
        s+= r.TIER2
        s+= r.TIER3
        s+= r.TIER4
        s+= r.TIER5
        est = [r.TIER1/s, r.TIER2/s, r.TIER3/s, r.TIER4/s, r.TIER5/s]
        return findmax(est)
    else return string(institution_id," ",tiers, "  not found")
    end
    
end

get_confidence (generic function with 1 method)

In [40]:
println(name_to_oid("Ohio State University", institution_mapping))
#println(get_confidence(NUM, name_to_oid("Unversity of British Columbia", institution_mapping), get_statement))
get_confidence(5, 465, get_statement)

465


Row,name,institution_id,TIER1,TIER2,TIER3,TIER4,TIER5
,String?,Int32?,Dec64,Dec64,Dec64,Dec64,Dec64


In [42]:
for j in 1:NUM
    println("********")
    println("Tier $j:")
    tier = Dict{}()
    for (i, type) in enumerate(t_allocation)
        if type == j
            
            #push!(tier, (institutions[i] => get_confidence(NUM, name_to_oid(institutions[i], institution_mapping), get_statement)))
            push!(tier, institutions[i] => 1)
            println(institutions[i])
            #println("  ", institutions[i], " Confidence: ", get_confidence(NUM, name_to_oid(institutions[i], institution_mapping), get_statement))
        end
    end
    #pretty_table(tier; header = ["University", "Confidence"])
    #pretty_table(tier; header = ["University", "Confidence"], backend=Val(:latex))
end

********
Tier 1:
University of Wisconsin, Madison
Stanford University
Duke University
University of Maryland
Northwestern University
London School of Economics and Political Science
Harvard University
Yale University
University of Michigan
Columbia University
Massachusetts Institute of Technology
New York University
Princeton University
University of Chicago
University of Pennsylvania
University of California, Berkeley
University of California Los Angeles (UCLA)
********
Tier 2:
Boston College
University of Warwick
National University of Singapore
Bocconi University
Ohio State University
University of Cambridge
University of Texas at Austin
Vanderbilt University
Erasmus University Rotterdam
Johns Hopkins University
University College London
University of Bonn
Arizona State University
University of Rochester
University of Virginia
Cornell University
University of British Columbia
Maastricht University
University of Washington
University of Oxford
Michigan State University
École d'Économ